# Uncented Kalman Filter


**Resources**

1) Chapter 14 of `Kalman Filter from Ground Up`; author Alex Becker; https://www.kalmanfilter.net

    a) the concept is introduced more or less as a series of recipes 

2) `A general method for approximating nonlinear transformations of probability distributions`; authors: Simon Julier, Jeffrey K. Uhlmann

   a) provides a fairly readable introduction into the concept of `sigma points` and their propagation through nonlinear system functions / observation functions.

3) `The unscented Kalman filter for nonlinear estimation`; authors: Eric A. Wan, Rudolph van der Merwe


4) `Unscented Kalman filter tutorial`; authors: Gabriel A. Terejanu

----

## System function

$$
\mathbf{x}_{k+1} = \mathbf{f}\left(\mathbf{x}_{k},\ \mathbf{u}_{k+1},\ \mathbf{v}_{k+1} \right)
$$

|  term / expression | description | dimension |
|--------------------|-------------|-----------|
| $\mathbf{f}( )$   | system function; in general a  nonlinear function | |
| $\mathbf{x}_{k}$  | previous state vector  | $n \times 1$ |
| $\mathbf{u}_{k+1}$ | driving input vector | tbd, but let us assume $n \times 1$ |
| $\mathbf{v}_{k+1}$ | noise vector | $q \times 1$ |
| $\mathbf{x}_{k+1}$  | current state vector  | $n \times 1$ |

The system function presented here is fairly general. It has been copied from `A general method for approximating nonlinear transformations of probability distributions`. Other resources omit the input signal $\mathbf{u}_{k+1}$ (eg. `The unscented Kalman filter for nonlinear estimation`).

In this general form the system function allows for nonlinear transformation of the state vector, the input vector and the process noise vector.

## Observation function / Measurement function

$$
\mathbf{z}_{k+1} = \mathbf{h}\left( \mathbf{x}_{k+1} \right) + \mathbf{w}_{k+1}
$$

|  term / expression | description | dimension |
|--------------------|-------------|-----------|
| $\mathbf{h}( )$   | observation / measurement function; in general a  nonlinear function | |
| $\mathbf{x}_{k+1}$  | current state vector  | $n \times 1$ |
| $\mathbf{z}_{k+1}$ | measurement vector | tbd, but let us assume $m \times 1$ |
| $\mathbf{w}_{k+1}$ | measurement noise vector | $m \times 1$ |

Covariance matrices of noise processes:

$$\begin{align}
E\left(\mathbf{v}_{i} \cdot \mathbf{v}_{i}^T \right) &= \mathbf{Q}_{i}\\
E\left(\mathbf{w}_{i} \cdot \mathbf{w}_{i}^T \right) &= \mathbf{R}_{i}\\
E\left(\mathbf{w}_{i} \cdot \mathbf{v}_{i}^T \right) &= \mathbf{0}\\
\end{align}
$$

## Augmented State vector

The concatenation of state vector $\mathbf{x}_{k}$ with process noise vector $\mathbf{v}_{k}$ is the `augmented` state vector $\mathbf{x}^a_{k}$.

$$
\mathbf{x}^a_{k} = \left[\begin{array}{c}
\mathbf{x}_{k} \\
\mathbf{v}_{k}
\end{array}\right]
$$

For the expectation of the augmented vector we get:

$$
E\left(\mathbf{x}^a_{k} \right) = \left[\begin{array}{c}
\mathbf{\hat{x}}_{k} \\
\mathbf{0}
\end{array}\right]
$$

$E\left(\mathbf{x}^a_{k} \right) $ is a $(n+q] \times 1$ vector.

For the covariance matrix $\mathbf{P}^a_k $ we obtain:

$$\begin{align}
\mathbf{P}^a_k &= E\left(  \left(\mathbf{x}^a_{k} - E\left(\mathbf{x}^a_{k} \right) \right) \cdot \left(\mathbf{x}^a_{k} - E\left(\mathbf{x}^a_{k} \right) \right)^T   \right) \\
&= E\left( \left[\begin{array}{c}
\mathbf{x}_{k} - \mathbf{\hat{x}}_{k} \\
\mathbf{v}_{k}
\end{array}\right] \cdot \left[\begin{array}{c}
\mathbf{x}_{k} - \mathbf{\hat{x}}_{k} \\
\mathbf{v}_{k}
\end{array}\right]^T \right) \\
&= \left[\begin{array}{cc}
E\left( \left(\mathbf{x}_{k} - \mathbf{\hat{x}}_{k} \right) \cdot \left(\mathbf{x}_{k} - \mathbf{\hat{x}}_{k} \right)^T \right) & E\left(  \left(\mathbf{x}_{k} - \mathbf{\hat{x}}_{k} \right) \cdot \mathbf{v}_{k}^T  \right) \\
E\left(  \mathbf{v}_{k} \cdot \left(\mathbf{x}_{k} - \mathbf{\hat{x}}_{k} \right)^T  \right) & E\left(  \mathbf{v}_{k} \cdot \mathbf{v}_{k}^T  \right)
\end{array}\right] \\
&= \left[\begin{array}{cc}
\mathbf{P}_{x,x}(k) & \mathbf{P}_{x,v}(k) \\
\mathbf{P}_{v,x}(k)   & \mathbf{Q}(k)
\end{array}\right]
\end{align}
$$

Here we used:

$$\begin{align}
\mathbf{P}_{x,x}(k) &= E\left( \left(\mathbf{x}_{k} - \mathbf{\hat{x}}_{k} \right) \cdot \left(\mathbf{x}_{k} - \mathbf{\hat{x}}_{k} \right)^T \right)\\
\mathbf{P}_{x,v}(k) &= E\left(  \left(\mathbf{x}_{k} - \mathbf{\hat{x}}_{k} \right) \cdot \mathbf{v}_{k}^T  \right)\\
\mathbf{P}_{v,x}(k) &= E\left(  \mathbf{v}_{k} \cdot \left(\mathbf{x}_{k} - \mathbf{\hat{x}}_{k} \right)^T  \right) \\
\mathbf{Q}(k) &= E\left(  \mathbf{v}_{k} \cdot \mathbf{v}_{k}^T  \right)
\end{align}
$$

## Computation of Sigma Points / Input

Input sigma points are computed from the factorisation of scaled covariance matrix $s \cdot \mathbf{P}^a_k$. The scaling factor $s$ is chosen as 

$$
s = n+q+\kappa
$$

$$
(n+q+\kappa) \cdot \mathbf{P}^a_k = \sqrt{(n+q+\kappa) \cdot \mathbf{P}^a_k} \cdot \sqrt{(n+q+\kappa) \cdot \mathbf{P}^a_k}^T
$$

Matrix $\sqrt{(n+q+\kappa) \cdot \mathbf{P}^a_k}$ is obtained from a `Cholesky` decomposition. Assuming that $\sqrt{(n+q+\kappa) \cdot \mathbf{P}^a_k}$ is the lower triangular matrix of such a decomposition we define $2 \cdot (n+q)$ column vectors $\mathbf{\sigma}^a_i(k)$ :

$$
\mathbf{\sigma}^a_i(k) = \begin{cases}
+\left(\sqrt{(n+q+\kappa) \cdot \mathbf{P}^a_k}\right)_{i}  & i'th \ column \ vector & 1 \le i \le n+q \\
-\left(\sqrt{(n+q+\kappa) \cdot \mathbf{P}^a_k}\right)_{i - (n+q)} & (i - (n+q) \ 'th \ column \ vector & (n+q+1) \le i \le 2 \cdot (n+q)
\end{cases}
$$

**i=0**

$$
\mathbf{\chi}_0(k,k) = E\left(\mathbf{x}^a_{k} \right) = \left[\begin{array}{c}
\mathbf{\hat{x}}_{k} \\
\mathbf{0}
\end{array}\right]
$$

**i > 0**

$$
\mathbf{\chi}_i(k,k) = \mathbf{\sigma}^a_i(k)  + E\left(\mathbf{x}^a_{k} \right)
$$

Using the definition of sigma points $\mathbf{\chi}_i(k,k)$ the covariance matrix $\mathbf{P}^a_k$ may be computed from the first of the following equations. The other equations just demonstrate that it is indeed the covariance matrix.

$$\begin{align}
\mathbf{P}^a_k &= \frac{1}{2\cdot (n+q+\kappa)} \cdot \sum_{i=1}^{2(n+q)} \left(\mathbf{\chi}_i(k,k) - E\left(\mathbf{x}^a_{k}\right) \right) \cdot \left(\mathbf{\chi}_i(k,k) - E\left(\mathbf{x}^a_{k}\right)^T \right) \\
&= \frac{1}{2\cdot (n+q+\kappa)} \cdot \sum_{i=1}^{2(n+q)} \mathbf{\sigma}^a_i(k) \cdot \mathbf{\sigma}^a_i(k)^T \\
&= \frac{1}{2\cdot (n+q+\kappa)} \cdot 2 \cdot  \sum_{i=1}^{(n+q)} \mathbf{\sigma}^a_i(k) \cdot \mathbf{\sigma}^a_i(k)^T \\
&= \frac{1}{2\cdot (n+q+\kappa)} \cdot 2 \cdot  \sum_{i=1}^{(n+q)} \left(\sqrt{(n+q+\kappa) \cdot \mathbf{P}^a_k}\right)_{i} \cdot \left(\sqrt{(n+q+\kappa) \cdot \mathbf{P}^a_k}\right)_{i}^T \\
&= \frac{2\cdot (n+q+\kappa)}{2\cdot (n+q+\kappa)} \cdot \sum_{i=1}^{(n+q)} \left(\sqrt{\mathbf{P}^a_k}\right)_{i} \cdot \left(\sqrt{\mathbf{P}^a_k}\right)_{i}^T \\
&= \mathbf{P}^a_k
\end{align}
$$

---

## Nonlinear transformation of sigma points

The nonlinear system function $\mathbf{f}( \ )$ transforms sigma points $\mathbf{\chi}_i(k,k) \; \ 0 \le i \le 2(n+q)$ into predicted sigma points $\mathbf{\chi}_i(k+1,k)$.

$$
\mathbf{\chi}_i(k+1,k) = \mathbf{f}\left(\mathbf{\chi}_i(k,k),\ \mathbf{u}_{k+1} \right)
$$

From the predicted sigma points the predicted mean and the predicted covariance matrix are computed.

**predicted mean**

$$
\mathbf{\hat{x}}(k+1, k) = \frac{1}{n+q+\kappa} \left(\kappa \cdot \mathbf{\chi}_0(k+1,k) + \frac{1}{2} \sum_{i=1}^{2(n+q)} \mathbf{\chi}_i(k+1,k) \right)
$$

**predicted covariance**

$$
\mathbf{P}(k+1,k) = \frac{1}{n+q+\kappa} \left(\kappa \left(\mathbf{\chi}_0(k+1,k) - \mathbf{\hat{x}}(k+1, k) \right) \cdot \left(\mathbf{\chi}_0(k+1,k) - \mathbf{\hat{x}}(k+1, k) \right)^T + \frac{1}{2} \sum_{i=1}^{2(n+q)}  \left(\mathbf{\chi}_i(k+1,k) - \mathbf{\hat{x}}(k+1, k) \right) \cdot \left(\mathbf{\chi}_i(k+1,k) - \mathbf{\hat{x}}(k+1, k) \right)^T \right)
$$

---

## Observation / Measurement


The predicted sigma points $\mathbf{\chi}_i(k+1,k)$ are propagated through the observation / measurement function $\mathbf{h}( \ )$ to yield sigma points $Z_i(k+1,k)$.


$$
Z_i(k+1,k) = \mathbf{h}\left(\mathbf{\chi}_i(k+1,k) \right) 
$$


The mean value of the observation is computed from these sigma points:

$$
\mathbf{\hat{z}}(k+1,k) = \frac{1}{n+q+\kappa} \left( \kappa \cdot Z_0(k+1,k) + \frac{1}{2} \sum_{i=1}^{2(n+q)}  Z_i(k+1,k) \right)
$$

$$
\mathbf{P}_{z,z}(k+1,k) = \frac{1}{n+q+\kappa} \left(\kappa \left(Z_0(k+1,k) - \mathbf{\hat{z}}(k+1,k)\right) \cdot \left(Z_0(k+1,k) - \mathbf{\hat{z}}(k+1,k)\right)^T + \frac{1}{2} \sum_{i=1}^{2(n+q)}  \left(Z_i(k+1,k) - \mathbf{\hat{z}}(k+1,k)\right) \cdot \left(Z_i(k+1,k) - \mathbf{\hat{z}}(k+1,k) \right)^T \right)
$$

To account for measurement noise the covariance matrix $\mathbf{R}$ of measurement noise is added. In total the measurement covariance matrix is:

$$
\mathbf{P}_{v,v}(k+1,k)  = \mathbf{P}_{z,z}(k+1,k)  + \mathbf{R}
$$

---